# Ejercicio Flask API
Para este ejercicio tendrás que desplegar un modelo de machine learning en una API para su consumo en tu entorno Local. Entrenarás un modelo, lo guardarás entrenado, y desarrollarás una API que permita consumir dicho modelo desde tu LocalHost.

**Se presenta el siguiente caso de uso**

Una empresa distribuidora de ámbito nacional pretende utilizar un modelo desarrollado por el departamento de data science, con el que consiguen una predicción de las ventas a partir de los gastos en marketing de anuncios en televisión, radio y periódicos. Quieren incorporar estos datos dentro de su página web interna, donde comparten todo tipo de información relativa a resultados de la empresa, ventas, adquisiciones, etc... La web está desarrollada en AngularJS, mientras que el modelo se desarrolló en Python, por lo que precisamos de una interfaz de comunicación entre ambos sistemas.

El equipo de desarrollo necesita que implementes un microservicio para que ellos puedan consumir el modelo desde la propia web. El microservicio tiene que cumplir las siguientes características:
1. Ofrezca la predicción de ventas a partir de todos los valores de gastos en publicidad.<br>(/api/v1/predict)
2. Posibilidad de reentrenar de nuevo el modelo con los posibles nuevos registros que se recojan y que devuelva la métrica de evaluación MSE. (/api/v1/retrain)


**NOTAS**: 
1. Deberás entregarlo en un repositorio de github
2. El repositorio github deberá contener todos los archivos necesarios para hacer una despliegue en railway.
3. Ojo con la ruta para hacer el load de tu modelo, comprueba cual es la ruta en la que está buscándolo.
4. Cuentas con un script de Python (*train.py*) con el código de entrenamiento del modelo ya hecho, puesto que el desarrollo de un modelo de machine learning no es el objetivo del ejercicio, sino el diseño de una API con Flask.

**Entregable**: fichero tu_nombre.json con {"repository":"repositorio de tu github"}

## Carga de librerias y lectura de datos

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

pd.set_option('display.max_columns', None)

In [41]:
df = pd.read_csv('./data/Advertising.csv')
df

,Unnamed: 0,TV,radio,newspaper,sales
0,0,230.1,37.8,69.2,22100.0
1,1,44.5,39.3,45.1,10400.0
2,2,17.2,45.9,69.3,9300.0
3,3,151.5,41.3,58.5,18500.0
4,4,180.8,10.8,58.4,12900.0
...,...,...,...,...,...
195,195,38.2,3.7,13.8,7600.0
196,196,94.2,4.9,8.1,9700.0
197,197,177.0,9.3,6.4,12800.0
198,198,283.6,42.0,66.2,25500.0


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  200 non-null    int64  
 1   TV          200 non-null    float64
 2   radio       200 non-null    float64
 3   newspaper   200 non-null    float64
 4   sales       200 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 7.9 KB


In [43]:
df = df.drop(['Unnamed: 0'], axis=1) # eliminamos columnas que no son de utilidad
df

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22100.0
1,44.5,39.3,45.1,10400.0
2,17.2,45.9,69.3,9300.0
3,151.5,41.3,58.5,18500.0
4,180.8,10.8,58.4,12900.0
...,...,...,...,...
195,38.2,3.7,13.8,7600.0
196,94.2,4.9,8.1,9700.0
197,177.0,9.3,6.4,12800.0
198,283.6,42.0,66.2,25500.0


## Modelo machine learning: regresión lineal

In [44]:
X = df[['TV', 'radio', 'newspaper']]
y = df['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

print('MSE: %.2f' % mean_squared_error(y_test, y_pred))
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

MSE: 3174097.35
Coefficient of determination: 0.90


In [46]:
pickle.dump(model, open('model_ADC.pkl', 'wb'))

In [47]:
# Abrir el archivo pickle con el protocolo no compatible
# para cargar archivos pickle que se crearon con versiones antiguas de Python 
#y utilizan un protocolo de serialización antiguo
with open('model_ADC.pkl', 'rb') as f:
    model = pickle.load(f, encoding='latin1')

# Guardar el modelo en un nuevo archivo utilizando un protocolo compatible
# para guardar archivos pickle que son compatibles con versiones antiguas de Python.
with open('model_ADC.pkl', 'wb') as f:
    pickle.dump(model, f, protocol=2)

In [48]:
loaded_model = pickle.load(open('model_ADC.pkl', 'rb'))

In [49]:
loaded_model.score(X_test, y_test)

0.899438024100912